In [ ]:
What is Mini-Batch Gradient Descent?

    Mini-Batch Gradient Descent is a hybrid between:
    Batch Gradient Descent (BGD) — uses all data per update
    Stochastic Gradient Descent (SGD) — uses one sample per update
    In Mini-Batch GD, we update weights using a small batch of samples (say 32, 64, or 128) at a time.

you update after a small batch

In [1]:

from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


In [2]:
X,y = load_diabetes(return_X_y=True)
print(X.shape)
print(y.shape)


(442, 10)
(442,)


In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)
reg = LinearRegression()
reg.fit(X_train,y_train)
LinearRegression()
print(reg.coef_)
print(reg.intercept_)

[  -9.15865318 -205.45432163  516.69374454  340.61999905 -895.5520019
  561.22067904  153.89310954  126.73139688  861.12700152   52.42112238]
151.88331005254167


In [4]:
y_pred = reg.predict(X_test)
r2_score(y_test,y_pred)


0.4399338661568968

In [10]:
import random

class MBGDRegressor:
    
    def __init__(self,batch_size,learning_rate=0.01,epochs=100):
        
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
    def fit(self,X_train,y_train):
        # init your coefs
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range(self.epochs):
            
            for j in range(int(X_train.shape[0]/self.batch_size)):
                
                idx = random.sample(range(X_train.shape[0]),self.batch_size)
                
                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                #print("Shape of y_hat",y_hat.shape)
                intercept_der = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.lr * intercept_der)

                # coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                coef_der = (-2 / self.batch_size) * np.dot((y_train[idx] - y_hat), X_train[idx])

                self.coef_ = self.coef_ - (self.lr * coef_der)
        
        print(self.intercept_,self.coef_)
    
    def predict(self,X_test):
        return np.dot(X_test,self.coef_) + self.intercept_


In [11]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)
mbr.fit(X_train,y_train)

y_pred = mbr.predict(X_test)
r2_score(y_test,y_pred)



155.70382541896493 [ 51.33197428  -1.26758321 163.18681687 126.66000852  42.21909919
  26.08751459 -96.53801263  95.34284755 159.15524036  86.98863911]


0.3213812888501586

In [12]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)
batch_size = 35

for i in range(100):
    
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])
sgd.coef_

 

array([  38.00973945,  -78.84913603,  349.2081088 ,  219.9225299 ,
         32.37124641,  -26.00691389, -160.98179744,  127.58418167,
        327.80319994,  120.14340123])

In [13]:
sgd.intercept_


array([155.0353734])

In [14]:

y_pred = sgd.predict(X_test)
r2_score(y_test,y_pred)


0.4337258108698532